# Backtest and Analysis Notebook

In [1]:
# Cell 0: Setup Python Path to Include 'src'
import os, sys

# Adjust path to ensure 'src' is importable
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [2]:
# Cell 1: Setup and Imports
import pandas as pd
import numpy as np
from src.data_loader import load_minute_underlying, fetch_daily_options
from src.strategy import generate_signals
from src.analytics import compute_pnl
from src.minute_simulator import simulate_minute_chain
import matplotlib.pyplot as plt
import time

ModuleNotFoundError: No module named 'src'

In [ ]:
symbol = 'SPY'
start = '2025-07-01'
end = '2025-07-02'
r = 0.01

In [ ]:
print("Fetching minute-level underlying data...")
u_df = load_minute_underlying(symbol, start, end)
u_df.to_csv("data/minute_underlying.csv", index=False)
print("Saved to data/minute_underlying.csv")

In [ ]:
print("Fetching daily option chain...")
option_chain = fetch_daily_options(symbol)
front_month = option_chain.sort_values('expiry').groupby('option_symbol').first().reset_index()
front_month.to_csv("data/option_chain.csv", index=False)
print("Saved to data/option_chain.csv")

In [ ]:
print("Simulating minute-level option chain...")
from tqdm import tqdm

minute_rows = []
for _, row in tqdm(front_month.iterrows(), total=front_month.shape[0]):
    for _, u_row in u_df.iterrows():
        minute_rows.append({
            'datetime': u_row['datetime'],
            'option_symbol': row['option_symbol'],
            'strike': row['strike'],
            'expiry': row['expiry'],
            'option_type': row['option_type'],
            'underlying_price': u_row['underlying_price'],
            'mid': np.nan,
            'bid': np.nan,
            'ask': np.nan
        })

minute_df = pd.DataFrame(minute_rows)
minute_df.to_csv("data/minute_simulated_chain.csv", index=False)
print("Saved to data/minute_simulated_chain.csv")

In [ ]:
df = generate_signals(minute_df, r=r, vol_mode='historical', filter_front_month=False)
df.to_csv("data/signals.csv", index=False)
print("Signals saved to data/signals.csv")

In [ ]:
pnl_df = compute_pnl(df)
pnl_df.to_csv("data/pnl.csv", index=False)
print("PnL saved to data/pnl.csv")

In [ ]:
# Cell 8: Visualize Results
plt.figure(figsize=(10, 5))
plt.plot(pnl_df['datetime'], pnl_df['cumulative_pnl'], label='Cumulative PnL')
plt.xlabel("Date")
plt.ylabel("PnL")
plt.title("Cumulative PnL Over Time")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()